In [1]:
import numpy as np
import random
import math

In [2]:
#list = np.arange(5)
#random_element = random.choice(list)

#random_number = random.randint(1, 10)

In [3]:
# Given the network topology
# We have the number of nodes "numNodes" = len(g) #g is a graph
numNodes = 14

In [5]:
class CR:
    def __init__(self, s, d, sl, status):     # Constructor : Gets called as soon as an object is declared
        self.s = s
        self.d = d
        self.sl = sl    # SL can be hign, mediuum, low
        self.status = status    # Status can be Allocatted, blocked, waiting

    def update_status(self, status) :    # A method to update the status
        status_list = ["allocated", "waiting", "blocked"]
        if status in status_list :
            self.status = status
        else :
            print("\n Invalid Status")

    def display_info(self):
        print(f"CR: Source Node ={self.s}, Destination Node={self.d}, Security Level={self.sl}, status={self.status}")
 

    @classmethod
    def generate_crs(cls, numCR = 1):            # cls = class, it's the same as self, except unlike __init__, it's not limited to an instance
        CRs = []
        sl_values = ["high", "medium", "low"]
        n = math.ceil(numCR/3)
        uniform_sample = n*sl_values     # Creates an array with n copies of sl_values
        
        for i in range(numCR):
            # Generating source, destination) pair
            #(s, d) =  random_nodes(numCR)
            s = random.randint(1, numNodes)
            d = random.randint(1, numNodes)   

            while s == d :
                d = random.randint(1, numNodes)  

            # Generating sl            
            sl = random.choice(uniform_sample)    # Randomly selecting from a uniform sample
            uniform_sample.remove(sl)  
            
            # Giving initial status
            status = "initialized"

            cr = cls(s, d, sl, status)
            CRs.append(cr)
        return CRs


#CRs = CR.generate_crs(n)

# Print the array of CR instances
#for cr in CRs:
 #   print(f"CR: s={cr.s}, d={cr.d}, sl={cr.sl}, status={cr.status}")


The above class CR has a constructor to initialize the values to its object. It has a class method "generate_crs" which takes the numCR parameter to generate that many CRs. the (s, d) pair is selected randomly. The sl is derived from a uniform list; and is allocated randomly to individual CR. 

In [6]:
X = 10    # Let's take the number of CRs to be 10
CRs = CR.generate_crs(X)

In [9]:
for i in range(10):
    CRs[i].display_info()

CR: Source Node =11, Destination Node=8, Security Level=low, status=initialized
CR: Source Node =4, Destination Node=5, Security Level=high, status=initialized
CR: Source Node =7, Destination Node=13, Security Level=medium, status=initialized
CR: Source Node =11, Destination Node=9, Security Level=medium, status=initialized
CR: Source Node =4, Destination Node=12, Security Level=low, status=initialized
CR: Source Node =13, Destination Node=3, Security Level=low, status=initialized
CR: Source Node =14, Destination Node=12, Security Level=medium, status=initialized
CR: Source Node =14, Destination Node=12, Security Level=high, status=initialized
CR: Source Node =10, Destination Node=6, Security Level=medium, status=initialized
CR: Source Node =9, Destination Node=6, Security Level=high, status=initialized
